# Using Slurm

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
os.chdir('../..')

In [5]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    project="takco",
    cores=8,
    memory="50 GB",
    scheduler_options={"dashboard_address": ":37659"},
)
cluster.scale(jobs=5)
cluster

SLURMCluster('tcp://130.37.199.8:37132', workers=0, threads=0, memory=0 B)

In [6]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://130.37.199.8:37132 Dashboard: http://130.37.199.8:37659/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [8]:
import logging
logging.getLogger().setLevel(logging.INFO)

import takco
conf = takco.Config('resources/config-chartedIn.toml')
pipe = takco.Config('resources/pipelines/chartedIn.toml')
pipe['step'][1]['split_compound_columns'] = False

takco.TableSet.run(pipe, force_step=3, **conf, executor=takco.DaskHashBag)

INFO:root:Running pipeline 'chartedIn' in output/chartedIn/
INFO:root:Chaining pipeline step 0-extract: {'step': 'extract', 'source': {'wiki': {'pred': 'http://www.wikidata.org/prop/direct/P2291', 'encoding': 'utf-8'}}, 'tables': [], 'workdir': 'output/chartedIn/', 'datadir': 'output/chartedIn/', 'resourcedir': 'resources', 'assets': [{'name': '0-extract', 'class': 'Annotation'}, {'name': '1-reshape', 'class': 'Annotation'}, {'name': '2-cluster', 'class': 'Annotation'}, {'name': '3-link', 'class': 'Annotation'}, {'name': '4-integrate', 'class': 'Annotation'}], 'kbs': [{'name': 'wikidata2019-P31inf-wikibase-simple-qualifiers', 'class': 'RDFSearcher', 'typeProperties': ['http://www.wikidata.org/prop/direct/P31'], 'qualifierIDProperty': 'http://www.wikidata.org/prop/direct/id', 'store': {'class': 'Trident', 'configuration': '../kb/wikidata/wikidata2019-P31inf-wikibase-simple-qualifiers/', 'ent_baseuri': 'http://www.wikidata.org/entity/', 'prop_baseuri': 'http://www.wikidata.org/prop/direc

In [15]:
tables = takco.HashBag._load('output/chartedIn/3-link/*')
tables = sorted(tables, key=lambda x: -x.get('numDataRows'))
takco.preview(tables)